# Network Friendly Recommendations Project: 1st assignment

As mentioned in class, this assignment is basically "part 1" of your final project assignment. The idea is to implement a simple(r) version of the problem that can be solved with "exact" methods (i.e., no approximations based on neural networks or other methods) that we learned in the first lectures about MDP and Q-learning methods. I provide below my suggestions for implementing this first version of your project:

##Libraries:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os, time, copy, math, random
import matplotlib.cm as cm

##Functions:

In [2]:
# Print u matrix with colors based on relations
def print_matrix(matrix,u_min):
    RED = '\033[91m'  # ANSI escape code for red text
    YELLOW = '\033[93m'  # ANSI escape code for yellow text
    RESET = '\033[0m'  # ANSI escape code to reset the text color

    for i, row in enumerate(matrix):
        for j, element in enumerate(row):
            if i == j:
                print(f"{YELLOW}{element:.3f}{RESET}", end=" ")  # Print diagonal element in yellow
            elif element < u_min:
                print(f"{RED}{element:.3f}{RESET}", end=" ")  # Print in red if smaller than min_value
            else:
                print(f"{element:.3f}", end=" ")
        print()

# Create a symmetric matrix
def create_symmetric_matrix(K):
    matrix = [[random.random() if i != j else 0 for j in range(K)] for i in range(K)]

    # Make the matrix symmetric by copying the upper triangle to the lower triangle
    for i in range(K):
        for j in range(i+1, K):
            matrix[j][i] = matrix[i][j]

    return matrix

# Choose which C of the K items will be cached
def random_cached_items(K, C):
    reward = [-1] * K  # Create a vector of length K with all 0 elements

    # Select C random indices
    indices = random.sample(range(K), C)

    # Set the selected indices to True
    for index in indices:
        reward[index] = 0

    return reward

# Random recommendation for current content watched
def random_recommendation(K, N, curr_content):
    recom = []  # Create an empty vector

    for _ in range(N):
        random_number = random.randint(0, K-1)
        while random_number == curr_content or random_number in recom:  # Check if random number is equal to curr_content or is already in recom
            random_number = random.randint(0, K-1)  # Generate a new random number
        recom.append(random_number)
    recom.sort()
    return recom

# Are all the recommended videos relevant to the current content being watched?
def all_relevant(N, curr_content, u, recom, u_min):
    all_relevant = True
    for i in range(N):
        if u[curr_content][recom[i]] < u_min: # Check if at least one is irrelevant
            all_relevant = False                # If one is irrelevant return false
            break
    return all_relevant

# User chooses the next video to watch
def user_chooses(K, N, q, u, u_min, a, recom, curr_content):
    if random.uniform(0, 1) > q:
        if all_relevant(N, curr_content, u, recom, u_min):  # If all recommended are relevant
            if random.uniform(0, 1) < a:  # User chooses one of the recommended
                new_content = recom[random.randint(0, N-1)]
            else:
                new_content = random.choice([x for x in range(K) if x != curr_content])
        else:                                               # If at least one recommended isn't relevant
            new_content = random.choice([x for x in range(K) if x != curr_content])
    else:
        new_content = -1
    return new_content

# Environment probability of user moving to next content given that he watches current content
# and he is being recommended the list recom
def env_prob(K, N, u, u_min, a, recom, curr_content, next_content):
    if curr_content==next_content:  # No possibility that the user watches the same content consequently
        env_prob = 0.0
    else:
        if all_relevant(N, curr_content, u, recom, u_min):  # If all recommended are relevant
        #print("All Relevant")
            if next_content in recom:     # If the next content was recommended
                env_prob = a/N + (1-a)/(K-1)
            else:                         # If the next content wasn't recommended
                env_prob = (1-a)/(K-1)
        else:                           # If at least one recommended isn't relevant
            #print("Not Relevant")
            env_prob = 1/(K-1)
    return env_prob

# All possible recommendations for state s
def possible_recom(K, N, s):
    items = list(range(K))
    items.remove(s)  # Remove 's' from the list of items

    def generate_combinations(curr_set, remaining_items):
        if len(curr_set) == N:
            return [curr_set]

        all_combinations = []
        for i, item in enumerate(remaining_items):
            new_set = curr_set + [item]
            new_remaining = remaining_items[i+1:]
            all_combinations.extend(generate_combinations(new_set, new_remaining))

        return all_combinations

    combinations = generate_combinations([], items)
    return list(map(list, combinations))

# All possible recommendations for all states
def all_states_possible_recom(K, N):

    all_combinations = []
    for s in range(K):
        state_combination = possible_recom(K, N, s)
        all_combinations.append(state_combination)

    return all_combinations

# Run "sessions_num" Monte Carlo episodes and compute the mean loss and the total loss
def monte_carlo_sessions(sessions_num, policy, reward, K, N, u_min, a, q, u):
    print("> Running Monte Carlo sessions...")
    total_loss = 0
    content_watched = 0
    for _ in range(sessions_num):
        curr_content = random.randint(0, K-1) # The first item viewed is random

        while True:
            recom = policy[curr_content]  # Recommend N items based on the policy
            curr_content = user_chooses(K, N, q, u, u_min, a, recom, curr_content)
            if curr_content == -1:
                break
            if reward[curr_content]==-1:
                total_loss += 1
            content_watched+=1

    if content_watched == 0:
        mean_loss = 0
    else:
        mean_loss = total_loss/content_watched

    return mean_loss, total_loss

# Find all the values above u_min
def find_values_above_min(matrix, u_min):
    result = []
    for i, row in enumerate(matrix):
        row_result = []
        for j, value in enumerate(row):
            if value > u_min:
                row_result.append(j)
        result.append(row_result)
    return result

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

def create_matrix(data):
    K = len(data)
    N = len(data[0])

    matrix = [[None] * N for _ in range(K)]

    for i in range(K):
        matrix[i] = list(data[i])

    return matrix

def create_tuple_list(matrix):
    K = len(matrix)
    N = len(matrix[0])

    tuple_list = []

    for i in range(K):
        rounded_values = [int(value) for value in matrix[i]]
        tuple_list.append(rounded_values)

    return tuple_list


### Environment (Content Catalogue):
- $K$ content items. I would pick $K \leq 100$ initially, as the code might get pretty slow otherwise.
- For every pair of items, $i$ and $j$, create a random value $u_{ij}$ in $[0,1]$ that indicates how related content $j$ is to content $i$. This is basically a random array $U$ of size $K \times K$ (you can choose to make it symmetric or not). Assume the diagonal of this array (i.e., all elements $u_{ii}$) are equal to 0 (to avoid recommending the same item just watched).
- Assume there is a threshold $u_{\text{min}}$ in $[0,1]$ below which two contents are "irrelevant" (this is an input parameter to play around with and observe its impact).
- Assume $C$ out of these contents are cached (use values of $C \leq 0.2K$). Cached items have cost 0, and non-cached have cost 1.

In [3]:
K = 20      # Number of content items
u_min = 0.5 # Minimum below which two contents are "irrelevant"
C = int(np.floor(0.2*K))   # Number of cached content items
u = create_symmetric_matrix(K)        # Create the matrix of relativity
reward = random_cached_items(K, C) # Create a vector that checks if the given item is cached
print_matrix(u,u_min)
print(reward)

0.000 0.394 0.153 0.128 0.308 0.799 0.297 0.457 0.084 0.582 0.977 0.203 0.788 0.012 0.320 0.333 0.973 0.892 0.379 0.314 
0.394 0.000 0.076 0.185 0.465 0.246 0.724 0.603 0.127 0.926 0.748 0.535 0.964 0.415 0.057 0.097 0.576 0.614 0.722 0.766 
0.153 0.076 0.000 0.907 0.122 0.690 0.906 0.807 0.451 0.949 0.665 0.312 0.505 0.264 0.846 0.268 0.237 0.055 0.750 0.297 
0.128 0.185 0.907 0.000 0.824 0.801 0.810 0.667 0.159 0.660 0.418 0.435 0.297 0.375 0.345 0.189 0.557 0.808 0.790 0.879 
0.308 0.465 0.122 0.824 0.000 0.159 0.401 0.605 0.336 0.259 0.138 0.077 0.164 0.553 0.875 0.491 0.291 0.446 0.798 0.110 
0.799 0.246 0.690 0.801 0.159 0.000 0.357 0.049 0.409 0.091 0.400 0.279 0.998 0.420 0.363 0.590 0.453 0.920 0.916 0.747 
0.297 0.724 0.906 0.810 0.401 0.357 0.000 0.272 0.522 0.145 0.916 0.332 0.845 0.591 0.369 0.987 0.621 0.749 0.297 0.592 
0.457 0.603 0.807 0.667 0.605 0.049 0.272 0.000 0.295 0.647 0.739 0.170 0.922 0.554 0.742 0.088 0.993 0.204 0.733 0.728 
0.084 0.127 0.451 0.159 0.336 0.

###Environment (User Model):

- A user might watch multiple items, one after the other during a session.
- After a user watches a content, $N = 2$ new items are recommended.
- With probability $q$ (input parameter): she ends the viewing session (i.e., does not watch another video).
- With probability $1-q$, she proceeds with one more video as follows:
  - If ALL $N$ recommended are "relevant" (i.e., have higher $u_{ij}$ than $u_{\text{min}}$), then
    - With probability $\alpha$ (input parameter): the user picks one of the $N$ recommended items (with equal probability).
    - With probability $1-\alpha$: the user picks any item $k$ from the entire catalogue of ${K}$ items with probability $p_k$ (you can assume for simplicity that $p_k = {\frac{1}{K}}$, i.e., uniform).
  - If at least one of the $N$ recommendations is irrelevant, then again the user picks any item $k$ from the entire catalogue of $K$ items with probability $p_k$.

In [4]:
N = 2         # Number of recommended content items
q = 0.05      # Probability of a session ending
a = 1         # Probability of choosing a recommended content item
round = 0     # Number of content item viewed during this session
history = []  # The history of content items viewed during this session
curr_content = random.randint(0, K-1) # The first item viewed is random
history.append(curr_content)  # Append first item in history

while True:
    #print("Round: "+str(round))
    #print("Current content: "+str(curr_content))
    #### THIS WILL BE REPLACED BY OUR ALGORITHMS ####
    recom = random_recommendation(K, N, curr_content)  # Recommend N random items
    #################################################
    #print("Recommendation: "+str(recom))
    curr_content = user_chooses(K, N, q, u, u_min, a, recom, curr_content)
    if curr_content == -1:
        break
    history.append(curr_content)
    round+=1

print(history)

[1, 6, 10, 5, 15, 9, 18, 16, 7, 14, 5, 6, 9, 4, 19, 8, 14, 10, 17, 15, 13, 8, 18, 14, 16, 6]


### Control variables and objective:
- Your algorithm must learn/optimize: for every possible item $i$ the user
might watch, a tuple of $N$ items to recommend.
- Objective: minimize the average total cost of items watched during a
session.

### Algorithms to implement:
1. Policy Iteration to optimize recommendations, assuming all environment
parameters are known.

### Policy evaluation
In general:

$V_{i+1}(s)=\color{yellow}{\sum_{a\in\mathcal{A}}\pi(a|s)}\left(\mathcal{R}_s^a+\gamma \sum_{s'\in\mathcal{S}}\mathcal{P}_{ss'}^{a}V_i(s')\right)$

Where:
  - $\color{yellow}{\text{One action per state in every policy so it isn't needed}}$
  - $\pi(a|s)$ is the set of actions that are allowed in this policy given state s
  - $\mathcal{R}_s^a$ is the reward of state s
  - $\gamma$ is equal to $1-q$
  - $\mathcal{P}_{ss'}^{a}$ is the environment probability of moving to state $s'$ given we are at state $s$ and we make action $a$
  - $V_i(s')$ is the last iteration's value function value of the next state

We could write it like this:

 $V_{i+1}(s)=\pi(a|s)\left(\mathcal{R}_s+(1-q) \sum_{s'\in\mathcal{S}}\mathcal{P}_{ss'}^aV_i(s')\right)$

The values of $\mathcal{P}_{ss'}^{a}$ are computed like this:

### Policy Improvement

$q_{\pi}(s,a) = \mathcal{R}_{s}+\sum_{s'\in\mathcal{S}}\mathcal{P}_{ss'}^aV_{\pi}(s')$

$\pi'(s)=argmax_{a\in \mathcal{A}}(q_{\pi}(s,a))$


In [5]:
def policy_iteration(K, N, u, u_min, a, q, reward, initial_policy):
    policy = []
    if initial_policy == 0:
        policy = [random_recommendation(K, N, i) for i in range(K)]
    else:
        policy = copy.deepcopy(initial_policy)                      # Initial random policy
    V = np.zeros(K)                                                 # Initial value function
    Q = np.zeros((K,int(math.comb(K-1,N))) # Initial Q function
    sessions_num = 100000   # Number of episodes run on Monte Carlo
    mean_loss = []
    total_loss = []

    def policy_evaluation(policy):
        print("> Evaluating policy...")
        threshold = 1e-10
        i=0
        while True:
            max_diff = 0
            for s in range(K):
                old_V = V[s]
                V[s] = reward[s] + (1-q) * sum(env_prob(K, N, u, u_min, a, policy[s], s, s_next) * V[s_next] for s_next in range(K))  # Calculate the value function for the current policy
                max_diff = max(max_diff, np.abs(old_V-V[s]))
                i+=1
            if max_diff < threshold:    # If the max_diff is lower than the threshold convergence is reached
                break

    def policy_improvement(policy):
        print("> Improving policy...")
        for s in range(K):   # For all states
            i=0
            all_recom = possible_recom(K, N, s)
            for recom in all_recom:                 # For all possible recommendations of this state
                Q[s][i] = reward[s] + (1-q) * sum(env_prob(K, N, u, u_min, a, recom, s, s_next) * V[s_next] for s_next in range(K))
                i+=1
            policy[s] = all_recom[np.argmax(Q[s])]  # Calculate the new policy by picking the best action for each state

    iteration = 0
    monte_carlo_elapsed_time = 0
    start_time = time.time()
    while True:
        last_policy = list(policy)
        #print("\n--------------------------------------------")
        #print("----------------- Policy "+str(iteration)+" -----------------")
        #print("--------------------------------------------\n")
        #print(policy)
        monte_carlo_start_time = time.time()
        mean_loss_i = 0
        total_loss_i = 0
        #mean_loss_i, total_loss_i = monte_carlo_sessions(sessions_num, policy, reward, K, N, u_min, a, q, u)
        monte_carlo_end_time = time.time()
        monte_carlo_elapsed_time += monte_carlo_end_time - monte_carlo_start_time
        mean_loss.append(mean_loss_i)
        total_loss.append(total_loss_i)
        #print("Mean Loss "+str(iteration)+":")
        #print(mean_loss[-1])
        policy_evaluation(policy)
        #print("Value Function "+str(iteration)+":")
        #print(V)
        policy_improvement(policy)
        iteration+=1
        if last_policy == policy or iteration == 30:                # If two consequent policies are the same break
            print("\033[91mSAME POLICY!\033[0m")
            break
    end_time = time.time()
    elapsed_time = end_time - start_time - monte_carlo_elapsed_time
    print("Iterations needed: "+ str(iteration))
    print("Optimal Policy: ")
    print(policy)
    policy_evaluation(policy)
    print("Optimal Value Function "+str(iteration))
    print(V)
    print("Mean Losses for each iteration: ")
    print(mean_loss)

    return policy, mean_loss, iteration, elapsed_time

## Running a toy example
$K$ = 10, $N$ = 2, $u_{min}$ = 0.5, $a$ = 1, $q$=0.05

In [6]:
toy_K = 10
toy_N = 2
toy_u = [[0.0, 0.9, 0.1, 0.1, 0.9, 0.1, 0.9, 0.9, 0.9, 0.1],
         [0.9, 0.0, 0.9, 0.9, 0.1, 0.9, 0.9, 0.1, 0.1, 0.1],
         [0.1, 0.9, 0.0, 0.1, 0.9, 0.1, 0.9, 0.1, 0.1, 0.1],
         [0.1, 0.9, 0.1, 0.0, 0.1, 0.9, 0.1, 0.1, 0.9, 0.1],
         [0.9, 0.1, 0.9, 0.1, 0.0, 0.9, 0.1, 0.1, 0.9, 0.1],
         [0.1, 0.9, 0.1, 0.9, 0.9, 0.0, 0.1, 0.9, 0.1, 0.1],
         [0.9, 0.9, 0.9, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.9],
         [0.9, 0.1, 0.1, 0.1, 0.1, 0.9, 0.1, 0.0, 0.1, 0.9],
         [0.9, 0.1, 0.1, 0.9, 0.9, 0.1, 0.1, 0.1, 0.0, 0.9],
         [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.9, 0.9, 0.9, 0.0]]
relations = find_values_above_min(toy_u, u_min)
for i in range(toy_K):
    print("State: "+str(i)+" -> "+str(relations[i]))
toy_u_min = 0.5
toy_a = 1
toy_q = 0.05
toy_reward = [0, 0, -1, -1, -1, -1, -1, -1, -1, -1]
opt_toy_policy, mean_loss_toy, iteration_toy, elapsed_time_toy = policy_iteration(toy_K, toy_N, toy_u, toy_u_min, toy_a, toy_q, toy_reward, 0)
print(elapsed_time_toy)

State: 0 -> [1, 4, 6, 7, 8]
State: 1 -> [0, 2, 3, 5, 6]
State: 2 -> [1, 4, 6]
State: 3 -> [1, 5, 8]
State: 4 -> [0, 2, 5, 8]
State: 5 -> [1, 3, 4, 7]
State: 6 -> [0, 1, 2, 9]
State: 7 -> [0, 5, 9]
State: 8 -> [0, 3, 4, 9]
State: 9 -> [6, 7, 8]
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
SAME POLICY!
Iterations needed: 5
Optimal Policy: 
[[1, 6], [0, 6], [1, 6], [1, 5], [0, 2], [1, 4], [0, 1], [0, 5], [0, 4], [0, 1]]
> Evaluating policy...
Optimal Value Function 5
[-6.44067797 -6.44067797 -7.44067797 -7.70081594 -7.59364407 -7.66630297
 -7.11864407 -7.70081594 -7.66630297 -7.94223687]
Mean Losses for each iteration: 
[0, 0, 0, 0, 0]
0.5273065567016602


## Experimenting with $a$ values
$a$ = [0.25, 0.5, 0.75, 1]

In [ ]:
mean_loss_list = []
iterations_list = []
elapsed_time_list = []
a_list = [0.25, 0.5, 0.75, 1]
initial_policy = [random_recommendation(K, N, i) for i in range(K)]
for a in a_list:
    opt_policy, mean_loss, iteration, elapsed_time = policy_iteration(K, N, u, u_min, a, q, reward, initial_policy)
    mean_loss_list.append(mean_loss)
    iterations_list.append(iteration)
    elapsed_time_list.append(elapsed_time)
print(mean_loss_list)
print(iterations_list)

In [ ]:
cmap = cm.get_cmap('summer')
plt.title("Policy Iteration Performance based on a")
plt.ylim(0, 1)
plt.xlabel("Iteration")
plt.ylabel("Mean Loss")
for i in range(len(a_list)):
    plt.plot(list(range(iterations_list[i])),
             mean_loss_list[i], color=cmap(i/len(a_list)),
             label='a='+str(a_list[i]))
    plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]+0.01,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(a_list)))
plt.legend()
plt.show
#plt.savefig("parameter_a_iteration.png", dpi=300)

## Experimenting with $N$ values
$N$ = [2, 3, 4]

In [ ]:
mean_loss_list = []
iterations_list = []
elapsed_time_list = []
a = 0.8
N_list = [2, 3, 4]
for N in N_list:
    opt_policy, mean_loss, iteration, elapsed_time = policy_iteration(K, N, u, u_min, a, q, reward, 0)
    mean_loss_list.append(mean_loss)
    iterations_list.append(iteration)
    elapsed_time_list.append(elapsed_time)
print(mean_loss_list)
print(iterations_list)

In [ ]:
cmap = cm.get_cmap('summer')
plt.title("Policy Iteration Performance based on Ν")
plt.ylim(0, 1)
plt.xlabel("Iteration")
plt.ylabel("Mean Loss")
for i in range(len(N_list)):
    plt.plot(list(range(iterations_list[i])),
             mean_loss_list[i], color=cmap(i/len(N_list)),
             label='N='+str(N_list[i]))
    plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]+0.01,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(N_list)))
plt.legend()
plt.show
#plt.savefig("parameter_N_iteration.png", dpi=300)
print(elapsed_time_list)

## Experimenting with $K$ values
$K$ = [20, 40, 60, 80, 100]

In [ ]:
mean_loss_list = []
iterations_list = []
elapsed_time_list = []
N = 2
K_list = [20, 40, 60, 80, 100]
u_local = create_symmetric_matrix(K_list[-1])
for K_local in K_list:
    C_local = int(np.floor(0.2*K_local))
    reward_local = random_cached_items(K_local, C_local)
    opt_policy, mean_loss, iteration, elapsed_time = policy_iteration(K_local, N, u_local, u_min, a, q, reward_local, 0)
    mean_loss_list.append(mean_loss)
    iterations_list.append(iteration)
    elapsed_time_list.append(elapsed_time)
print(mean_loss_list)
print(iterations_list)

In [ ]:
cmap = cm.get_cmap('summer')
plt.title("Policy Iteration Performance based on K")
plt.ylim(0, 1)
plt.xlabel("Iteration")
plt.ylabel("Mean Loss")
for i in range(len(K_list)):
    plt.plot(list(range(iterations_list[i])),
             mean_loss_list[i], color=cmap(i/len(K_list)),
             label='K='+str(K_list[i]))
    if i == 1:
        plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]+0.04,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(K_list)))
    else:
        plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]+0.01,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(K_list)))
plt.legend()
plt.show
#plt.savefig("parameter_K_iteration.png", dpi=300)
print(elapsed_time_list)

## Experimenting with $u_{min}$ values
$u_{min}$ = [0.2, 0.4, 0.6, 0.8]

In [ ]:
mean_loss_list = []
iterations_list = []
elapsed_time_list = []
u_min_list = [0.2, 0.4, 0.6, 0.8]
for u_min_local in u_min_list:
    opt_policy, mean_loss, iteration, elapsed_time = policy_iteration(K, N, u, u_min_local, a, q, reward, 0)
    mean_loss_list.append(mean_loss)
    iterations_list.append(iteration)
    elapsed_time_list.append(elapsed_time)
print(mean_loss_list)
print(iterations_list)

In [ ]:
cmap = cm.get_cmap('summer')
plt.title("Policy Iteration Performance based on $u_{min}$")
plt.ylim(0, 1)
plt.xlabel("Iteration")
plt.ylabel("Mean Loss")
for i in range(len(u_min_list)):
    plt.plot(list(range(iterations_list[i])),
             mean_loss_list[i], color=cmap(i/len(u_min_list)),
             label='$u_{min}$='+str(u_min_list[i]))
    if i == 1:
        plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]-0.04,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(u_min_list)))
    else:
        plt.text(iterations_list[i]-1.5,mean_loss_list[i][1]+0.01,str(int(elapsed_time_list[i]*1000))+" ms",color=cmap(i/len(u_min_list)))
plt.legend()
plt.show
#plt.savefig("parameter_u_min_iteration.png", dpi=300)
print(elapsed_time_list)

## Running an example with the default values


In [ ]:
opt_policy, mean_loss, iteration, elapsed_time = policy_iteration(K, N, u, u_min, a, q, reward, 0)

2. Q-learning, assuming parameters $α$ and $u_{min}$ are not known (but the $u_{ij}$
relevance values, and $q$ are still known).

### Q-Learning Algorithm

- Initially $Q_0(s,a)=0$ $\forall s,a$
- Choose a random initial state
- while True:
  - sample action $a$
  - get next state $s'$
  - if $s'$ is terminal:
    - $target=\mathcal{R}_s$
    - sample new random initial state $s'$
  - else:
    - $target = \mathcal{R}_s+(1-q) \max _{a'}Q(s',a')$
  - $Q_{i+1}(s,a)=(1-\mathfrak{a})Q_i(s,a)+\mathfrak{a}\cdot target$
  - $s=s'$

####How to sample action $a$?
$\epsilon$-Greedy
- with probability $\epsilon(s,t)$
  - randomly choose $a$
- with probability $1-\epsilon(s,t)$
  - $a=argmax_a Q_i(s,a)$


In [7]:
def q_learning(K, N, q, u, u_min, a, reward, opt_policy):
    # Initialize Q which is a list of dictionaries
    Q = [{} for _ in range(K)]
    opt_policy = create_matrix(opt_policy)  # Get the policy from policy iteration as a matrix
    policy = np.zeros((K,N))
    old_policy = np.zeros((K,N))
    num_possible_actions = int(math.factorial(K-1)/(math.factorial(N)*math.factorial(K-N-1)))   # Number of possible recommendations for each state
    for s in range(K):                          # For all states create the dictionary containing all recommendation tuples
        all_recoms = possible_recom(K, N, s)
        for recom in all_recoms:                # Initialize all values to 0
            Q[s][tuple(recom)]=0

    s = random.randint(0, K-1)  # Initial state
    epsilon = 1                 # Initial epsilon is 1
    round = 0                   # Round is the total number of user video choises
    epsilon_hist = []           # A history of all epsilon values
    round_hist = []             # The history of rounds
    alpha_hist = []             # The history of learning rate
    episode = 0                 # Episodes initially 0
    start_time = time.time()    # Start counting time
    alpha = 0.01

    while True:
        round += 1
        alpha_hist.append(alpha)
        epsilon_hist.append(epsilon)
        round_hist.append(round)
        # With probability epsilon get random recommendation
        if random.uniform(0, 1) < epsilon:
            recom = random_recommendation(K, N, s)
        # With probability 1-epsilon get the recommendation with the largest Q value
        else:
            recom = list(max(Q[s],key=Q[s].get))
        # User chooses the new state
        new_s = user_chooses(K, N, q, u, u_min, a, recom, s)
        # If the new state is terminal
        if new_s == -1:
            # Compute the new policy
            for i in range(K):
                recom_tuple = (list(max(Q[i],key=Q[i].get)))
                for j in range(N):
                    policy[i][j] = recom_tuple[j]
            # Every 100 episodes
            if episode % 100 == 0:
                if episode == 0: print(str(policy.tolist()), end='')
                #else:  print("\r" + str(policy.tolist()), end='')
                # If the old policy from 100 episodes before stayed the same break
                if np.array_equal(policy, old_policy):
                    print("\r" + str(policy.tolist()), end='')
                    break
                # Assign new policy to the old
                old_policy = np.array(policy)
            # If the policy is equal to the optimal break
            if np.array_equal(policy, opt_policy):
                print("\r" + str(policy.tolist()), end='')
                break
            # Get the new initial state
            new_s = random.randint(0, K-1)
            episode += 1
            # Calculate the new epsilon
            epsilon = 0.01 + (1 - 0.01) * math.exp(-0.00005 * episode)
            # The new state becomes the current state
            s = new_s
            continue
        # Calculate the Q function's value
        Q[s][tuple(recom)] += alpha * ( reward[new_s] + (1-q) * max(Q[new_s].values()) - Q[s][tuple(recom)])
        s = new_s

    # Calculate the time elapsed
    end_time = time.time()
    time_elapsed = end_time - start_time

    policy = create_tuple_list(policy)
    return policy, episode, time_elapsed


## Running a toy example
$K$ = 10, $N$ = 2, $u_{min}$ = 0.5, $a$ = 1, $q$=0.05

In [9]:
toy_K = 10
toy_N = 2
toy_u = [[0.0, 0.9, 0.1, 0.1, 0.9, 0.1, 0.9, 0.9, 0.9, 0.1],
         [0.9, 0.0, 0.9, 0.9, 0.1, 0.9, 0.9, 0.1, 0.1, 0.1],
         [0.1, 0.9, 0.0, 0.1, 0.9, 0.1, 0.9, 0.1, 0.1, 0.1],
         [0.1, 0.9, 0.1, 0.0, 0.1, 0.9, 0.1, 0.1, 0.9, 0.1],
         [0.9, 0.1, 0.9, 0.1, 0.0, 0.9, 0.1, 0.1, 0.9, 0.1],
         [0.1, 0.9, 0.1, 0.9, 0.9, 0.0, 0.1, 0.9, 0.1, 0.1],
         [0.9, 0.9, 0.9, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.9],
         [0.9, 0.1, 0.1, 0.1, 0.1, 0.9, 0.1, 0.0, 0.1, 0.9],
         [0.9, 0.1, 0.1, 0.9, 0.9, 0.1, 0.1, 0.1, 0.0, 0.9],
         [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.9, 0.9, 0.9, 0.0]]
relations = find_values_above_min(toy_u, u_min)
for i in range(toy_K):
    print("State: "+str(i)+" -> "+str(relations[i]))
toy_u_min = 0.5
toy_a = 1
toy_q = 0.05
toy_reward = [0, 0, -1, -1, -1, -1, -1, -1, -1, -1]
opt_toy_policy_q, episodes_q, time_toy = q_learning(toy_K, toy_N, toy_q, toy_u, toy_u_min, toy_a, toy_reward, opt_toy_policy)
print()
print("Episodes needed to get to optimal policy: "+str(episodes_q))
print(opt_toy_policy)
print(opt_toy_policy_q)

State: 0 -> [1, 4, 6, 7, 8]
State: 1 -> [0, 2, 3, 5, 6]
State: 2 -> [1, 4, 6]
State: 3 -> [1, 5, 8]
State: 4 -> [0, 2, 5, 8]
State: 5 -> [1, 3, 4, 7]
State: 6 -> [0, 1, 2, 9]
State: 7 -> [0, 5, 9]
State: 8 -> [0, 3, 4, 9]
State: 9 -> [6, 7, 8]
[[1.0, 6.0], [0.0, 6.0], [1.0, 6.0], [1.0, 8.0], [0.0, 2.0], [1.0, 4.0], [0.0, 1.0], [0.0, 5.0], [0.0, 4.0], [2.0, 8.0]]
Episodes needed to get to optimal policy: 30900
[[1, 6], [0, 6], [1, 6], [1, 5], [0, 2], [1, 4], [0, 1], [0, 5], [0, 4], [0, 1]]
[[1, 6], [0, 6], [1, 6], [1, 8], [0, 2], [1, 4], [0, 1], [0, 5], [0, 4], [2, 8]]


In [ ]:
mean_loss_list_pi = []
mean_loss_list_q = []
episodes_list_q= []
a = 1.0
for K_local in range(10,35,3):
    sessions_num = 100000
    u_local = create_symmetric_matrix(K_local)
    C_local = int(np.floor(0.2*K_local))
    reward_local = random_cached_items(K_local, C_local)
    print("\033[91mPolicy Iteratation\033[0m"+" K = "+str(K_local))
    opt_policy_local, mean_loss_local, iteration_local, elapsed_time_local = policy_iteration(K_local, N, u_local, u_min, a, q, reward_local, 0)
    print("\033[91mQ-Learning\033[0m"+" K = "+str(K_local))
    q_policy_local, episodes_local, time_elapsed_local = q_learning(K_local, N, q, u_local, u_min, a, reward_local, opt_policy_local)
    print()
    mean_loss_q, total_loss_q = monte_carlo_sessions(sessions_num, q_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_pi, total_loss_pi = monte_carlo_sessions(sessions_num, opt_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_list_pi.append(mean_loss_pi)
    mean_loss_list_q.append(mean_loss_q)
    episodes_list_q.append(episodes_local)
    print(mean_loss_list_pi)
    print(mean_loss_list_q)

    cmap = cm.get_cmap('summer')

    plt.figure()
    plt.title("Policy Iteration vs Q-Learning Performance")
    plt.ylim(0, 1)
    plt.xlabel("K")
    plt.ylabel("Mean Loss")
    plt.plot(range(10,K_local+1,3), mean_loss_list_pi, color=cmap(0), label='Policy Iteration')
    plt.plot(range(10,K_local+1,3), mean_loss_list_q, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("pi_vs_q_performance_a_1_new.png", dpi=300)

    plt.figure()
    plt.title("Q-Learning Episodes Needed")
    plt.xlabel("K")
    plt.ylabel("Number of Episodes")
    plt.plot(range(10,K_local+1,3), episodes_list_q, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("q_episodes_a_1_new.png", dpi=300)


Policy Iteratation K = 10
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
SAME POLICY!
Iterations needed: 3
Optimal Policy: 
[[2, 6], [6, 7], [6, 7], [6, 7], [1, 2], [3, 6], [1, 2], [1, 2], [0, 1], [1, 6]]
> Evaluating policy...
Optimal Value Function 3
[-10.5        -10.25641026 -10.25641026 -10.25641026 -10.74358974
 -10.5         -9.74358974  -9.74358974 -10.76388889 -10.5       ]
Mean Losses for each iteration: 
[0, 0, 0]
Q-Learning K = 10
[[2.0, 6.0], [6.0, 7.0], [6.0, 7.0], [6.0, 7.0], [1.0, 2.0], [3.0, 6.0], [1.0, 2.0], [1.0, 2.0], [0.0, 1.0], [1.0, 6.0]]
> Running Monte Carlo sessions...
> Running Monte Carlo sessions...
[0.5013909857828405]
[0.5013727896460233]
Policy Iteratation K = 13
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
> Evaluating policy...
> Improving policy...
SAME POLICY!
Iterations need

In [ ]:
mean_loss_list_pi = []
mean_loss_list_q = []
episodes_list_q_8 = []
a = 0.8
for K_local in range(10,35,3):
    sessions_num = 100000
    u_local = create_symmetric_matrix(K_local)
    C_local = int(np.floor(0.2*K_local))
    reward_local = random_cached_items(K_local, C_local)
    print("\033[91mPolicy Iteratation\033[0m"+" K = "+str(K_local))
    opt_policy_local, mean_loss_local, iteration_local, elapsed_time_local = policy_iteration(K_local, N, u_local, u_min, a, q, reward_local, 0)
    print("\033[91mQ-Learning\033[0m"+" K = "+str(K_local))
    q_policy_local, episodes_local, time_elapsed_local = q_learning(K_local, N, q, u_local, u_min, a, reward_local, opt_policy_local)
    print()
    mean_loss_q, total_loss_q = monte_carlo_sessions(sessions_num, q_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_pi, total_loss_pi = monte_carlo_sessions(sessions_num, opt_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_list_pi.append(mean_loss_pi)
    mean_loss_list_q.append(mean_loss_q)
    episodes_list_q_8.append(episodes_local)
    print(mean_loss_list_pi)
    print(mean_loss_list_q)

    cmap = cm.get_cmap('summer')

    plt.figure()
    plt.title("Policy Iteration vs Q-Learning Performance")
    plt.ylim(0, 1)
    plt.xlabel("K")
    plt.ylabel("Mean Loss")
    plt.plot(range(10,K_local+1,3), mean_loss_list_pi, color=cmap(0), label='Policy Iteration')
    plt.plot(range(10,K_local+1,3), mean_loss_list_q, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("pi_vs_q_performance_a_0_8.png", dpi=300)

    plt.figure()
    plt.title("Q-Learning Episodes Needed")
    plt.xlabel("K")
    plt.ylabel("Number of Episodes")
    plt.plot(range(10,K_local+1,3), episodes_list_q_8, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("q_episodes_a_0_8.png", dpi=300)


In [ ]:
mean_loss_list_pi = []
mean_loss_list_q = []
episodes_list_q_6 = []
a = 0.6
for K_local in range(10,35,3):
    sessions_num = 100000
    u_local = create_symmetric_matrix(K_local)
    C_local = int(np.floor(0.2*K_local))
    reward_local = random_cached_items(K_local, C_local)
    print("\033[91mPolicy Iteratation\033[0m"+" K = "+str(K_local))
    opt_policy_local, mean_loss_local, iteration_local, elapsed_time_local = policy_iteration(K_local, N, u_local, u_min, a, q, reward_local, 0)
    print("\033[91mQ-Learning\033[0m"+" K = "+str(K_local))
    q_policy_local, episodes_local, time_elapsed_local = q_learning(K_local, N, q, u_local, u_min, a, reward_local, opt_policy_local)
    print()
    mean_loss_q, total_loss_q = monte_carlo_sessions(sessions_num, q_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_pi, total_loss_pi = monte_carlo_sessions(sessions_num, opt_policy_local, reward_local, K_local, N, u_min, a, q, u_local)
    mean_loss_list_pi.append(mean_loss_pi)
    mean_loss_list_q.append(mean_loss_q)
    episodes_list_q_6.append(episodes_local)
    print(mean_loss_list_pi)
    print(mean_loss_list_q)

    cmap = cm.get_cmap('summer')

    plt.figure()
    plt.title("Policy Iteration vs Q-Learning Performance")
    plt.ylim(0, 1)
    plt.xlabel("K")
    plt.ylabel("Mean Loss")
    plt.plot(range(10,K_local+1,3), mean_loss_list_pi, color=cmap(0), label='Policy Iteration')
    plt.plot(range(10,K_local+1,3), mean_loss_list_q, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("pi_vs_q_performance_a_0_6.png", dpi=300)

    plt.figure()
    plt.title("Q-Learning Episodes Needed")
    plt.xlabel("K")
    plt.ylabel("Number of Episodes")
    plt.plot(range(10,K_local+1,3), episodes_list_q_6, color=cmap(0.9), label='Q-Learning')
    plt.legend()
    #plt.savefig("q_episodes_a_0_6.png", dpi=300)


In [ ]:
plt.figure()
plt.title("Q-Learning Episodes Needed over a")
plt.xlabel("K")
plt.ylabel("Number of Episodes")
plt.plot(range(10,K_local+1,3), episodes_list_q_6, color=cmap(0.9), label='a=0.6')
plt.plot(range(10,K_local+1,3), episodes_list_q_8, color=cmap(0.5), label='a=0.8')
plt.plot(range(10,K_local+1,3), episodes_list_q, color=cmap(0.1), label='a=1.0')
plt.legend()
#plt.savefig("q_episodes_a.png", dpi=300)

In [ ]:
sessions_num = 100000
random_policy = [random_recommendation(K, N, i) for i in range(K)]
mean_loss_i, total_loss_i = monte_carlo_sessions(sessions_num, opt_policy, reward, K, N, u_min, a, q, u)
mean_loss_q, total_loss_q = monte_carlo_sessions(sessions_num, q_policy, reward, K, N, u_min, a, q, u)
mean_loss_r, total_loss_r = monte_carlo_sessions(sessions_num, random_policy, reward, K, N, u_min, a, q, u)
print(mean_loss_i)
print(mean_loss_q)
print(mean_loss_r)


###Experiments to try and report:
- Demonstrate that Policy Iteration indeed solves the problem optimally
(you could try a hand-picked, toy scenario for this, not necessarily randomly chosen).
- Show the average cost achieved for different parameters ($K$, $u_{min}$ , $\alpha$, $q$).
- Prove that your Q-learning algorithm correctly learns the optimal policy
as well (in the partially model-free environment above). Demonstrate the
convergence speed to the optimal (for different K values).
- Try to ”break” Q-learning. At what values of K does your PC or colab
code start going ”too slow”?